In [ ]:
import pandas as pd
import numpy as np
import csv
import bs4
import math

In [ ]:
biz = pd.read_csv('yelp_business.csv')
biz.head()

In [ ]:
biz_att = pd.read_csv('yelp_business_attributes.csv')
biz_att.head()

In [ ]:
biz_hrs = pd.read_csv('yelp_business_hours.csv')
biz_hrs.head()

In [ ]:
merge1 = biz.merge(biz_att, how='outer', on='business_id')

In [ ]:
final_df = merge1.merge(biz_hrs, how='outer', on='business_id')

In [ ]:
final_df.head(20)

In [ ]:
print(final_df.columns[:10],\
final_df.columns[10:20],\
final_df.columns[20:30],\
final_df.columns[30:40],\
final_df.columns[40:50],\
final_df.columns[50:60],\
final_df.columns[60:70],\
final_df.columns[70:80],\
final_df.columns[80:90],\
final_df.columns[90:100],\
final_df.columns[100:110],\
final_df.columns[110:])

In [ ]:
final_df['Ambience_romantic'].unique()

In [ ]:
final_df['monday'].unique()

In [ ]:
s = '7:30-17:0'

In [ ]:
lst = s.split(sep='-')
lst

In [ ]:
lst[0].split(sep=':')

In [ ]:
lst[1].split(sep=':')

In [ ]:
17-7+(0-30)/60

In [ ]:
cities = final_df['city'].value_counts()[:20].index
final_df['city'].value_counts()[:10]

In [ ]:
for city in cities:
    final_df[city] = final_df['city'].apply(lambda x: 1 if x == city else 0)

In [ ]:
city_lst = []
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
for city in cities:
    city_lst.append(city)
clst2 = []
for i in range(0, 10):
    print(cities[i], i)
    clst2.append(cities[i])
days_lst = []
for day in days:
    day2 = day + 'hrs'
    days_lst.append(day2)
ind_vars = days_lst + city_lst + ['review_count']
vals=['stars']+ind_vars

In [ ]:
df_use = pd.DataFrame(columns=vals)
len(final_df)

In [ ]:
import numbers
vals=['stars']+ind_vars
df3 = pd.DataFrame(columns=vals)
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
for idx, shop in final_df.iterrows():
    row = [shop['stars']]
    for day in days:
        if shop[day] != 'None' and shop[day] != None and shop[day] != 'Na' and shop[day] != 'nan' and \
        isinstance(shop[day], numbers.Number) == False:
            lst = shop[day].split(sep='-')
            start_time = lst[0]
            close_time = lst[1]
            start_times = start_time.split(sep=':')
            close_times = close_time.split(sep=':')
            start_hour = start_times[0]
            start_min = start_times[1]
            close_hour = close_times[0]
            close_min = close_times[1]
            hrs = float(close_hour) - float(start_hour) + (float(close_min) + float(start_min))/60
        else:
                hrs = 0
        row.append(hrs)
    for city in city_lst:
        row.append(shop[city])
    row.append(shop['review_count'])
    shop_df = pd.DataFrame([row], columns=vals)
    df3 = df3.append(shop_df, ignore_index=True)

In [ ]:
df3 = pd.read_csv('MACS30200Projectdata.txt')
import statsmodels.api as sm
y = df3['stars']
X = df3[['mondayhrs', 'tuesdayhrs', 'wednesdayhrs', 'thursdayhrs', 'fridayhrs', 'saturdayhrs', \
          'sundayhrs'] + clst2 + ['review_count']]
X = sm.add_constant(X)
lg = sm.OLS(y, X.astype(float), missing='drop').fit()
print(lg.summary())

In [ ]:
X2 = df3[['mondayhrs', 'tuesdayhrs', 'wednesdayhrs', 'thursdayhrs', 'fridayhrs', 'saturdayhrs', \
          'sundayhrs', 'review_count']]
X2 = sm.add_constant(X2)
lg2 = sm.OLS(y, X2.astype(float), missing='drop').fit()
print(lg2.summary())

In [ ]:
lg2

In [ ]:
import csv
df3.to_csv('/Users/apple/Documents/MACS30200Projectdata.txt')

In [ ]:
import requests
import time
import bs4
import json
import pandas as pd
import numpy as np

In [ ]:
cols = ['stars', 'review_count','mondayhrs', 'tuesdayhrs', 'wednesdayhrs', 'thursdayhrs', 'fridayhrs', 'saturdayhrs', 'sundayhrs']
def calc_hrs(str_start, str_close):
    #print(str_start, str_close)
    start_time = str_start.split(sep=':')
    close_time = str_close.split(sep=':')
    start_half = start_time[1][-2: ]
    close_half = close_time[1][-2: ]
    start_hour = start_time[0]
    start_min = start_time[1].strip(start_half).strip()
    close_hour = close_time[0]
    close_min = close_time[1].strip(close_half).strip()
    if start_half == 'pm' and close_half == 'pm':
        hrs = float(close_hour) - float(start_hour) + (float(close_min) + float(start_min))/60
    elif start_half == 'am' and close_half == 'pm':
        hrs = float(close_hour) + 12 - float(start_hour) + (float(close_min) + float(start_min))/60
    elif start_half == 'pm' and close_half == 'am' and close_hour == '12':
        hrs = float(close_hour) - float(start_hour) + (float(close_min) + float(start_min))/60
    elif start_half == 'pm' and close_half == 'am' and close_hour != 12:
        hrs = float(close_hour) + 12 - float(start_hour) + (float(close_min) + float(start_min))/60
    elif start_half == 'am' and close_half == 'am' and close_hour != 12:
        hrs = float(close_hour) + 24 - float(start_hour) + (float(close_min) + float(start_min))/60
    elif start_half == 'am' and close_half == 'am' and close_hour == 12:
        hrs = float(close_hour) + 12 - float(start_hour) + (float(close_min) + float(start_min))/60
    return hrs

def get_biz_info(url):
    print(url)
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    star_tag = soup.find_all('script', type="application/ld+json")
    d = json.loads(star_tag[0].text)
    stars = d['aggregateRating']['ratingValue']
    review_count = d['aggregateRating']["reviewCount"]
    tag_lst = soup.find_all('th', scope="row")
    am_lst = []
    pm_lst = []
    biz_row = [stars, review_count]
    #if url == 'https://www.yelp.com/biz/avec-chicago':
        #print('yes!')
        #biz_row = biz_row + [12, 12, 12, 12, 12, 10.5, 15]
    #elif url.endswith('brazilian-steakhouse-chicago-4'):
        #biz_row = biz_row + [8, 8, 8, 8, 8.5, 11, 9.5]
   # elif url.endswith('yuzu-sushi-and-robata-grill-chicago'):
        #biz_row = biz_row + 
    #else:
    for tag in tag_lst:
        tag_text = tag.next_sibling.next_sibling.text.strip().split("-")
        #print(tag_text)
        #print(tag_text[0], 'yes')
        dy = tag.next_sibling.parent.find_all('th', scope="row")
        if dy != []:
            if dy[0].text in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']:
                if len(tag_text) == 3:
                    start1 = tag_text[0].strip()
                    close2 = tag_text[2].strip()
                    for k in range(0, len(tag_text[1])):
                        if tag_text[1][k] == 'm':
                            close1 = tag_text[1][0:(k+1)].strip()
                            start2 = tag_text[1][(k+1): ].strip()
                            break
                    hrs1 = calc_hrs(start1, close1) + calc_hrs(start2, close2)
                    biz_row.append(hrs1)
                elif tag_text[0].strip().endswith('am') or tag_text[0].strip().endswith('pm'):
                    start1 = tag_text[0].strip()
                    close1 = tag_text[1].strip()
                    hrs2 = calc_hrs(start1, close1) 
                    biz_row.append(hrs2)
                elif tag_text[0] == 'Closed':
                    biz_row.append(0)
    biz_df = pd.DataFrame([biz_row], columns=cols)
    return biz_df    

def get_page_info(url):
    page_df = pd.DataFrame(columns=cols)
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    page_lst = soup.find_all('a', class_="biz-name js-analytics-click")
    for tag in page_lst:
        biz_url = 'https://www.yelp.com' + tag.get('href')
        biz_df = get_biz_info(biz_url)
        page_df = page_df.append(biz_df, ignore_index=True)
    return page_df
            
            
    
def get_df():
    tot_df = pd.DataFrame(columns=cols)
    for i in range(0, 200, 10):
        print(i)
        time.sleep(2)
        url = 'https://www.yelp.com/search?find_loc=Chicago,+IL&start={}'.format(i)
        page_df = get_page_info(url)
        tot_df = tot_df.append(page_df, ignore_index=True)
    return tot_df

In [ ]:
url = 'https://www.yelp.com/biz/avec-chicago'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.text, 'html.parser')
tag_lst = soup.find_all('th', scope="row")

In [ ]:
tag_lst[0].next_sibling.parent.find_all('th', scope="row")[0].text

In [ ]:
a[1].strip()

In [ ]:
url = 'https://www.yelp.com/biz/avec-chicago'
get_biz_info(url)

In [ ]:
df_web = get_df()

In [ ]:
df_web.head()

In [ ]:
df_web.to_csv('/Users/apple/Documents/Yelptop200.txt')

In [ ]:
df_web['const'] = pd.Series(1, index=df_web.index)
df_web.head()

In [ ]:
MSE_lst = []
for idx, shop in df_web.iterrows():
    predicted_stars = 3.4831*float(shop['const']) + (-0.0235)*float(shop['mondayhrs']) + \
    0.0127*float(shop['tuesdayhrs']) + 0.0221*float(shop['wednesdayhrs']) +\
    0.0122*float(shop['thursdayhrs']) + 0.0164*float(shop['fridayhrs']) + (-0.0099)*float(shop['saturdayhrs']) + \
    (-0.0180)*float(shop['sundayhrs']) + 0.0004*float(shop['review_count'])
    MSE = (predicted_stars - real_stars) ** 2
    MSE_lst.append(MSE)

In [ ]:
print(math.sqrt(sum(MSE_lst)/len(MSE_lst)), sum(MSE_lst)/len(MSE_lst))

In [ ]:
df_web.head()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
# This next command is specifically for Jupyter Notebook
star = final_df['stars']
%matplotlib notebook 
count, bins, ignored = plt.hist(star, 9, edgecolor='black', normed=True)
plt.title('Distribution of Yelp rating stars', fontsize=15)
plt.xlabel('Rating stars')
plt.ylabel('Portion of businesses')

In [ ]:
final_df['stars'].value_counts()

In [ ]:
final_df['stars'].unique()

References:  
1. https://stackoverflow.com/questions/33094056/is-it-possible-to-append-series-to-rows-of-dataframe-without-making-a-list-first  
2. https://stackoverflow.com/questions/12555323/adding-new-column-to-existing-dataframe-in-python-pandas  
3. Our CS122 Project from the last quarter.  